From https://habr.com/ru/company/wunderfund/blog/677068/

In [2]:
import pandas as pd

df = pd.DataFrame({
    "booking_id": [1001, 1002, 1003, 1004, 1005],
    "property" : ["A", "A", "B", "B", "C"],
    "created_at": ["2022-03-01", "2022-02-10", "2022-04-12",
                   "2022-04-11", "2022-06-05"],
    "checkin_date": ["2022-06-01", "2022-06-10", "2022-06-02",
                     "2022-06-20", "2022-08-10"],
    "checkout_date": ["2022-06-06", "2022-06-15", 
                      "2022-06-06","2022-06-28", "2022-08-16"],
    "amount": [5400, 5600, 4800, 9000, 6500]
})

# изменение типа данных 
date_cols = ["created_at","checkin_date","checkout_date"]

df[date_cols] = df[date_cols].astype("datetime64[ns]")

# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500


In [3]:
# создание столбца year_month
df["year_month"] = df["created_at"].dt.to_period("M")

# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-03
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06


Как прибавить к дате временной интервал?

In [4]:
df.loc[df["booking_id"]==1001, "checkout_date"] = \
df.loc[df["booking_id"]==1001, "checkout_date"] + \
pd.DateOffset(days=1)

# проверка результата
print(df.loc[df["booking_id"]==1001, "checkout_date"])

0   2022-06-07
Name: checkout_date, dtype: datetime64[ns]


Как найти интервал в днях между двумя датами?

In [5]:
df["checkout_date"][0] - df["checkin_date"][0]

Timedelta('6 days 00:00:00')

In [6]:
# разница, выраженная в днях
df["days_to_checkin"] = \
(df["checkin_date"] - df["created_at"]).dt.days

# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,days_to_checkin
0,1001,A,2022-03-01,2022-06-01,2022-06-07,5400,2022-03,92
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02,120
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04,51
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04,70
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06,66


Как сформировать последовательность дат между начальной и конечной датами?

In [7]:
# создание объекта DataFrame calendar
calendar = df[["property","checkin_date","checkout_date"]]

pd.date_range(calendar["checkin_date"][0], calendar["checkout_date"][0])

DatetimeIndex(['2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04',
               '2022-06-05', '2022-06-06', '2022-06-07'],
              dtype='datetime64[ns]', freq='D')

In [8]:
# создание столбца booked_days
calendar.loc[:, "booked_days"] = calendar.apply(
    
    lambda x: list(
        pd.date_range(
            x.checkin_date, 
            x.checkout_date - pd.DateOffset(days=1)
        ).date
    ),
    axis = 1
    
)
# вывод объекта DataFrame
calendar

/tmp/ipykernel_6411/3218262485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar.loc[:, "booked_days"] = calendar.apply(


,property,checkin_date,checkout_date,booked_days
0,A,2022-06-01,2022-06-07,"[2022-06-01, 2022-06-02, 2022-06-03, 2022-06-0..."
1,A,2022-06-10,2022-06-15,"[2022-06-10, 2022-06-11, 2022-06-12, 2022-06-1..."
2,B,2022-06-02,2022-06-06,"[2022-06-02, 2022-06-03, 2022-06-04, 2022-06-05]"
3,B,2022-06-20,2022-06-28,"[2022-06-20, 2022-06-21, 2022-06-22, 2022-06-2..."
4,C,2022-08-10,2022-08-16,"[2022-08-10, 2022-08-11, 2022-08-12, 2022-08-1..."


In [9]:
# функция explode 
calendar = calendar.explode(
    column="booked_days", ignore_index=True
)[["property","booked_days"]]
# вывод первых 5 строк
calendar.head()

,property,booked_days
0,A,2022-06-01
1,A,2022-06-02
2,A,2022-06-03
3,A,2022-06-04
4,A,2022-06-05
